In [ ]:
using Transformation, Plots, Logging
disable_logging(LogLevel(1000));

# Манипулятор KUKA KR-16
Манипулятор KUKA KR-16 отличается от упрощенной модели двумя дополнительными смещениями, перед второй кинематической парой и после третьей.

In [ ]:
l = [
    520, # подъем вдоль колонны
    160, # смещение вперед
    980, # длина плеча
    150, # смещение локтя
    860, # длина предплечья
    153  # длина кисти
];

## Решение прямой задачи кинематики для манипулятора KUKA KR-16

In [ ]:
function kr16(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    q = map(deg2rad, q)
    t0 = Transf()
    t1 = t0 + Transf(Vec(0, 0, l[1]), Quat(q[1], Vec(0, 0, 1)))
    t2 = t1 + Transf(Vec(0, l[2], 0), Quat(q[2], Vec(1, 0, 0)))
    t3 = t2 + Transf(Vec(0, 0, l[3]), Quat(q[3], Vec(1, 0, 0)))
    t4 = t3 + Transf(Vec(0, -l[4], 0))
    t5 = t4 + Transf(Vec(0, 0, l[5]), Quat(q[4], Vec(0, 0, 1)) * Quat(q[5], Vec(1, 0, 0)))
    t6 = t5 + Transf(Vec(0, 0, l[6]), Quat(q[6], Vec(0, 0, 1)))
    [t0, t1, t2, t3, t4, t5, t6]
end;

### Вспомогательные функции для отображения

In [ ]:
include("graphics.jl");

## Пример решения прямой задачи кинематики

In [ ]:
qs = [0, 45, -45, 0, 90, 0]
qe = [-120, -15, -135, 60, -90, 360]

pathX = []
pathY = []
pathZ = []
@gif for t = 0:.01:1
    p = plot(
        xlims = (-1000, 1000),
        ylims = (-1000, 1000),
        zlims = (0, 2000),
        size = (500, 500)
    )
    q = qs + t * (qe - qs)
    chain = kr16(l, q)
    lst = last(chain)
    push!(pathX, lst.v.x)
    push!(pathY, lst.v.y)
    push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, lst, 100)
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end

## Решение обратной задачи кинематики для манипулятора KUKA KR-16

In [ ]:
function angleTo(a::Vec, b::Vec, n::Vec)::Real
    c = cross(a, b)
    d = dot(a, b)
    s = dot(c, n) > 0 ? 1 : -1
    s * atan(norm(c), d)
end

function wrap360(a::Real)::Real
    (a + 540) % 360 - 180
end

function ik(l::Array{<:Real,1}, t::Transf, i::Array{<:Real,1}=[1, 1, 1])::Array{<:Real,1}
    fifth = t + Vec(0, 0, -l[4]) - Vec(0, 0, l[1])
    q1 = atan(fifth.y, fifth.x) - pi + pi / 2 * i[1]
    d = sqrt((sqrt(fifth.x^2 + fifth.y^2) - l[2] * i[1])^2 + fifth.z^2)
    l2 = l[3]
    l3 = sqrt(l[4]^2 + l[5]^2)
    q3a = -0.5pi + atan(l[5], l[4])
    q3 = q3a + i[1] * i[2] * acos((l2^2 + l3^2 - d^2)/(2 * l2 * l3)) + pi
    q2 = i[1] * (i[2] * acos((l2^2 + d^2 - l3^2)/(2 * l2 * d)) + atan(fifth.z, sqrt(fifth.x^2 + fifth.y^2) - l[2] * i[1]) - pi / 2)
    o = Quat(q1, Vec(0, 0, 1)) * Quat(q2, Vec(1, 0, 0)) * Quat(q3, Vec(1, 0, 0))
    xo = o * Vec(1, 0, 0)
    zo = o * Vec(0, 0, 1)
    xt = t.q * Vec(1, 0, 0)
    zt = t.q * Vec(0, 0, 1)
    xr = cross(zo, zt)
    q4 = angleTo(xo, xr, zo) + pi / 2 - pi / 2 * i[3]
    q5 = angleTo(zo, zt, xr) * i[3]
    q6 = angleTo(xr, xt, zt) + pi / 2 - pi / 2 * i[3]
    q = map(rad2deg, [q1, q2, q3, q4, q5, q6])
    map(wrap360, q)
end;

## Начальное и конечное положения

In [ ]:
s = Transf(Vec(0, -400, 800), Quat(pi / 3, Vec(1, 0, 0)))
e = Transf(Vec(400, 600, 1200), Quat(pi / 2, Vec(0, 1, 0)));

### Линейное движение

In [ ]:
path_x = []
path_y = []
path_z = []
target_x = []
target_y = []
target_z = []
@gif for t = 0:0.01:1
    target = Transf(
        s.v + t * (e.v - s.v),
        slerp(s.q, e.q, t),
    )
    q = ik(l, target, [1, 1, 1])
    p = plot(
        xlims = (-1000, 1000),
        ylims = (-1000, 1000),
        zlims = (0, 2000),
        size = (500, 500)
    )
    chain = kr16(l, q)
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z)
    push!(target_x, target.v.x); push!(target_y, target.v.y); push!(target_z, target.v.z);
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, s, 75)
    plotAxis!(p, e, 75)
    plotAxis!(p, lst, 100)
    plot!(p, path_x, path_y, path_z, color = :red, label = "path")
end

### Переброска

In [ ]:
pathX = []
pathY = []
pathZ = []
@gif for t = 0:0.01:1
    qs = ik(l, s)
    qe = ik(l, e)
    q = qs + t * (qe - qs)
    p = plot(
        xlims = (-1000, 1000),
        ylims = (-1000, 1000),
        zlims = (0, 2000),
        size = (500, 500)
    )
    chain = kr16(l, q)
    lst = last(chain)
    push!(pathX, lst.v.x)
    push!(pathY, lst.v.y)
    push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, s, 75)
    plotAxis!(p, e, 75)
    plotAxis!(p, lst, 100)
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end

### Влияние флагов конфигурации

In [ ]:
flags = [
    [1, 1, 1],
    [-1, 1, 1],
    [-1, -1, 1],
    [-1, -1, -1],
    [-1, 1, -1],
    [1, 1, -1],
    [1, -1, -1],
    [1, -1, 1],
    [1, 1, 1]
]
target = Transf(Vec(900, 600, 1500), Quat(pi / 2, Vec(0, 1, 0)));
@gif for i = 1:0.01:8.99
    p = plot(
        xlims = (-1000, 1000),
        ylims = (-1000, 1000),
        zlims = (0, 2000),
        size = (500, 500)
    )
    
    flagsStart = flags[Int(floor(i))]
    flagsEnd = flags[Int(floor(i+1))]
    t = i % 1
    arg = 0.5 - cos(t * pi) / 2
    qStart = ik(l, target, flagsStart)
    qEnd = ik(l, target, flagsEnd)
    q = qStart + arg * (qEnd - qStart)
    
    chain = kr16(l, q)
    lst = last(chain)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, lst, 100)
    plotAxis!(p, target, 100)
end